# IMPORT LIBRARIES

In [1]:
import pandas as pd
import random
from pyomo.environ import *
import numpy as np
import time
import matplotlib.pyplot as plt


# IMPORT DATA

In [2]:
df = pd.read_csv(r'C:\Users\Matteo.Gabellini\OneDrive - Alma Mater Studiorum Università di Bologna\DOTTORATO\1.RICERCA\0.CONFERENCE PAPER\6.ICIL\1.WAREHOUSE ALLOCATION\0.DATA\DatasetClean.csv')
df['Articolo'] = df['Articolo'].astype(str)
df

,Unnamed: 0,Num. Ordine,Mese-Giorno,Articolo,Descrizione,Pezzi ordinati,Pezzi evasi,Pz x CT,Pz x TH,Volume pezzo,Volume cartone,Ecr1,Ecr2,Ecr3,Ecr4,Canale,Cliente,PV,Percorso
0,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,33.96900,Cura Persona,Igienico Sanitari,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26
1,1,738379,04-18,3456,STUDIO L.5 INVISI FIX GEL FOR.LIQ.150ml,6,6,6,0,0.378000,2.66000,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
2,2,738379,04-18,7199,STUDIO L.8 FIX&FORCE GEL IPERFOR.150 ML.,6,6,6,0,0.303750,2.94400,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
3,3,738379,04-18,43556,STUDIO L.9 INDESTRUC.GEL ESTREMO 150 ML,6,6,6,0,0.720000,2.81600,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
4,4,738379,04-18,50045,STUDIO L.5 INVISI FIX GEL CR.FOR.VAS.150,6,6,6,0,0.405000,2.54375,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447292,2447292,791134,12-29,92720,CHANTECLAIR SGRASS.600 MLx2 PZ.LIMONE,1,1,6,0,3.271125,23.04000,Casa e Bucato,Superfici,Detergenti Superfici,Sgrassatori,B2C,2152842,UD,999
2447293,2447293,791134,12-29,98819,VIM GEL BAGNO 5in1 ANTICALCARE 1000 ML,1,1,12,0,1.559250,25.63600,Casa e Bucato,Bagno,Detergenti Bagno e WC,Detergenti Bagno,B2C,2152842,UD,999
2447294,2447294,791134,12-29,105877,CIF CREMA GREEN ACTIVE 500 ML PINK BLOOM,1,1,16,0,0.728000,17.02400,Casa e Bucato,Superfici,Detergenti Superfici,Detergenti Multiuso,B2C,2152842,UD,999
2447295,2447295,791134,12-29,107171,OMINO B.DET.IDROCAPS SALVAFIBRE 20pz,1,1,8,0,2.025000,20.46000,Casa e Bucato,Bucato,Detersivi Bucato,Detersivi Capsule Lavatrice,B2C,2152842,UD,999


In [3]:
df_vol = df.groupby('Articolo')[['Volume pezzo']].median().reset_index()
df_vol

,Articolo,Volume pezzo
0,100002,0.060000
1,100003,0.060000
2,100004,0.060000
3,100005,0.060000
4,100006,0.060000
...,...,...
18655,99972,0.059375
18656,99973,0.270000
18657,99974,3.312000
18658,99975,0.084000


In [4]:
df_stock = pd.read_excel(r'C:\Users\Matteo.Gabellini\OneDrive - Alma Mater Studiorum Università di Bologna\DOTTORATO\1.RICERCA\0.CONFERENCE PAPER\6.ICIL\1.WAREHOUSE ALLOCATION\0.DATA\Giacenza media articoli 2024.xlsx')
df_stock['ARTICOLO'] = df_stock['ARTICOLO'].astype(str)
df_stock = df_stock.groupby('ARTICOLO')[['GIACENZA MEDIA']].mean().reset_index()
df_stock

,ARTICOLO,GIACENZA MEDIA
0,100002,1057.6
1,100003,735.4
2,100004,1072.4
3,100005,1170.0
4,100006,1232.2
...,...,...
20919,99972,19.2
20920,99973,26.5
20921,99974,13.9
20922,99975,7.0


In [5]:
# Perform the join based on 'Articolo'
df_stock_vol = df_stock.merge(df_vol[['Articolo','Volume pezzo']], how='left', left_on='ARTICOLO', right_on='Articolo')

# Drop duplicate column 'ARTICOLO' after merge
df_stock_vol.drop(columns=['Articolo'], inplace=True)

#Compute stock in volum
df_stock_vol['Volume pezzo [m3]'] = df_stock_vol['Volume pezzo'] / 1000
df_stock_vol['Giacenza Pezzi Volume [m3]'] = df_stock_vol['GIACENZA MEDIA'] * df_stock_vol['Volume pezzo [m3]']

#df_stock_vol = df_stock_vol.drop_duplicates()

df_stock_vol

,ARTICOLO,GIACENZA MEDIA,Volume pezzo,Volume pezzo [m3],Giacenza Pezzi Volume [m3]
0,100002,1057.6,0.060000,0.000060,0.063456
1,100003,735.4,0.060000,0.000060,0.044124
2,100004,1072.4,0.060000,0.000060,0.064344
3,100005,1170.0,0.060000,0.000060,0.070200
4,100006,1232.2,0.060000,0.000060,0.073932
...,...,...,...,...,...
20919,99972,19.2,0.059375,0.000059,0.001140
20920,99973,26.5,0.270000,0.000270,0.007155
20921,99974,13.9,3.312000,0.003312,0.046037
20922,99975,7.0,0.084000,0.000084,0.000588


In [6]:
df_stock_vol['Giacenza Pezzi Volume [m3]'].sum()

9121.832503351417

In [7]:
# Perform a left join to maintain the original number of rows in df
df = df.merge(df_stock_vol[['ARTICOLO', 'Giacenza Pezzi Volume [m3]']], how='left', left_on='Articolo', right_on='ARTICOLO')

df['Volume evaso [m3]'] = df['Pezzi evasi'] * df['Volume pezzo'] / 1000

# Drop the extra 'ARTICOLO' column from df_stock_vol (after the merge)
df.drop(columns=['ARTICOLO'], inplace=True)

# Ensure no additional duplicates were introduced
#df = df.drop_duplicates()

df

,Unnamed: 0,Num. Ordine,Mese-Giorno,Articolo,Descrizione,Pezzi ordinati,Pezzi evasi,Pz x CT,Pz x TH,Volume pezzo,...,Ecr1,Ecr2,Ecr3,Ecr4,Canale,Cliente,PV,Percorso,Giacenza Pezzi Volume [m3],Volume evaso [m3]
0,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,...,Cura Persona,Igienico Sanitari,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26,3.474766,0.007728
1,1,738379,04-18,3456,STUDIO L.5 INVISI FIX GEL FOR.LIQ.150ml,6,6,6,0,0.378000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.206955,0.002268
2,2,738379,04-18,7199,STUDIO L.8 FIX&FORCE GEL IPERFOR.150 ML.,6,6,6,0,0.303750,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.140424,0.001823
3,3,738379,04-18,43556,STUDIO L.9 INDESTRUC.GEL ESTREMO 150 ML,6,6,6,0,0.720000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.419256,0.004320
4,4,738379,04-18,50045,STUDIO L.5 INVISI FIX GEL CR.FOR.VAS.150,6,6,6,0,0.405000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.148149,0.002430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447292,2447292,791134,12-29,92720,CHANTECLAIR SGRASS.600 MLx2 PZ.LIMONE,1,1,6,0,3.271125,...,Casa e Bucato,Superfici,Detergenti Superfici,Sgrassatori,B2C,2152842,UD,999,2.330022,0.003271
2447293,2447293,791134,12-29,98819,VIM GEL BAGNO 5in1 ANTICALCARE 1000 ML,1,1,12,0,1.559250,...,Casa e Bucato,Bagno,Detergenti Bagno e WC,Detergenti Bagno,B2C,2152842,UD,999,0.943970,0.001559
2447294,2447294,791134,12-29,105877,CIF CREMA GREEN ACTIVE 500 ML PINK BLOOM,1,1,16,0,0.728000,...,Casa e Bucato,Superfici,Detergenti Superfici,Detergenti Multiuso,B2C,2152842,UD,999,0.743142,0.000728
2447295,2447295,791134,12-29,107171,OMINO B.DET.IDROCAPS SALVAFIBRE 20pz,1,1,8,0,2.025000,...,Casa e Bucato,Bucato,Detersivi Bucato,Detersivi Capsule Lavatrice,B2C,2152842,UD,999,0.532170,0.002025


In [8]:
df.drop_duplicates(subset = 'Articolo').groupby('Ecr1')['Giacenza Pezzi Volume [m3]'].sum().sum()

8737.318431127718

In [9]:
# assuming your DataFrame is called df_all (or df)
cols_to_check = ["Ecr1", "Ecr2", "Ecr3", "Ecr4"]
df = df.dropna(subset=cols_to_check, how="any").reset_index(drop=True)
df

,Unnamed: 0,Num. Ordine,Mese-Giorno,Articolo,Descrizione,Pezzi ordinati,Pezzi evasi,Pz x CT,Pz x TH,Volume pezzo,...,Ecr1,Ecr2,Ecr3,Ecr4,Canale,Cliente,PV,Percorso,Giacenza Pezzi Volume [m3],Volume evaso [m3]
0,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,...,Cura Persona,Igienico Sanitari,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26,3.474766,0.007728
1,1,738379,04-18,3456,STUDIO L.5 INVISI FIX GEL FOR.LIQ.150ml,6,6,6,0,0.378000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.206955,0.002268
2,2,738379,04-18,7199,STUDIO L.8 FIX&FORCE GEL IPERFOR.150 ML.,6,6,6,0,0.303750,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.140424,0.001823
3,3,738379,04-18,43556,STUDIO L.9 INDESTRUC.GEL ESTREMO 150 ML,6,6,6,0,0.720000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.419256,0.004320
4,4,738379,04-18,50045,STUDIO L.5 INVISI FIX GEL CR.FOR.VAS.150,6,6,6,0,0.405000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.148149,0.002430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2298719,2447292,791134,12-29,92720,CHANTECLAIR SGRASS.600 MLx2 PZ.LIMONE,1,1,6,0,3.271125,...,Casa e Bucato,Superfici,Detergenti Superfici,Sgrassatori,B2C,2152842,UD,999,2.330022,0.003271
2298720,2447293,791134,12-29,98819,VIM GEL BAGNO 5in1 ANTICALCARE 1000 ML,1,1,12,0,1.559250,...,Casa e Bucato,Bagno,Detergenti Bagno e WC,Detergenti Bagno,B2C,2152842,UD,999,0.943970,0.001559
2298721,2447294,791134,12-29,105877,CIF CREMA GREEN ACTIVE 500 ML PINK BLOOM,1,1,16,0,0.728000,...,Casa e Bucato,Superfici,Detergenti Superfici,Detergenti Multiuso,B2C,2152842,UD,999,0.743142,0.000728
2298722,2447295,791134,12-29,107171,OMINO B.DET.IDROCAPS SALVAFIBRE 20pz,1,1,8,0,2.025000,...,Casa e Bucato,Bucato,Detersivi Bucato,Detersivi Capsule Lavatrice,B2C,2152842,UD,999,0.532170,0.002025


# STATISTICS

In [14]:
df.groupby('Mese-Giorno')['Num. Ordine'].nunique().describe()

count    366.000000
mean     196.710383
std       68.565443
min       10.000000
25%      146.250000
50%      197.500000
75%      246.000000
max      516.000000
Name: Num. Ordine, dtype: float64

In [15]:
df.groupby('Mese-Giorno')['Percorso'].nunique().describe()

count    366.000000
mean      25.336066
std        5.583161
min        2.000000
25%       22.000000
50%       26.000000
75%       29.000000
max       39.000000
Name: Percorso, dtype: float64

In [16]:
df['Articolo'].nunique()

16421

In [22]:
df_stock['GIACENZA MEDIA'].describe()

count    20924.000000
mean       279.597104
std       1060.029682
min          0.100000
25%         14.400000
50%         60.300000
75%        238.725000
max      85943.400000
Name: GIACENZA MEDIA, dtype: float64

In [21]:
df.groupby(['Articolo'])['Volume pezzo'].mean().describe()

count    16421.000000
mean         1.566121
std          9.737135
min          0.000000
25%          0.135000
50%          0.423500
75%          1.020000
max        460.512000
Name: Volume pezzo, dtype: float64